In [ ]:
# import dependencies
import tensorflow as tf
from glob import glob
import pandas as pd
import librosa
import librosa.display
import numpy as np
import soundfile as sf
from librosa.util import fix_length
import scipy as sp
import spafe
from scipy.stats import kurtosis,skew,mode,gstd,describe,iqr,gmean,hmean,median_abs_deviation,variation,tstd,gstd,moment,entropy
from spafe.features import pncc,gfcc,bfcc,mfcc,lfcc,lpc,ngcc,rplp,psrcc,msrcc
from sklearn.preprocessing import normalize

In [ ]:
pth=glob(r"C:\Users\Mohd Ashhad\OneDrive\Documents\MVD Augmented\*wav") # insert path of folder which contains all recordings

In [ ]:
x=pth[0]
print(x[54:])

In [ ]:
# fucntion that extracts features

from spafe.features.gfcc import gfcc
from spafe.features.mfcc import mfcc

def features_extractor(file):
    
    #load the file (audio)
    
    sig, sr = librosa.load(file)
    
    # pad audio files to avoid any dimensional issue
    
    required_audio_size=3
    audio = fix_length(sig, size=required_audio_size*sr) 
    
    #Code for feature extraction

    # local feature extraction (use only one feature as per requirement)
    
    S = gfcc(sig=sig, fs=sr, num_ceps=40,nfilts=128,nfft=2048,win_hop=0.0232,win_len=0.0464)
#     S = mfcc(sig=sig, fs=sr, num_ceps=40,nfilts=128,nfft=2048,win_hop=0.0232,win_len=0.0464)
#     S=librosa.feature.melspectrogram(sig, sr=sr, n_mels=128,n_fft=2048,hop_length=512,win_length=1024)

    # Global feature extraction
    
    ft=sp.fft.fft(sig) # code for computing the spectrum using Fast Fourier Transform
    magnitude=np.absolute(ft)
    spec=magnitude[0:11025]
    
    k=kurtosis(spec)
    s=skew(spec)
    mean=np.mean(spec)
    z=np.array(mode(spec)[0])
    mode_var=float(z)
    i=iqr(spec)
    g=gmean(spec)
    h=hmean(spec)
    dev=median_abs_deviation(spec)
    var=variation(spec)
    variance=np.var(spec)
    std=tstd(spec)
    gstd_var=gstd(spec)
    ent= entropy(spec)
    
    features=[mode_var,k,s,mean,i,g,h,dev,var,variance,std,gstd_var,ent]
    features=normalize([features])
    features=np.array(features)
    features=np.reshape(features,(13,))
    feat=[S,features] # save the matrix and vector in a list
    
    return feat


In [ ]:
# Function for building the dataframe and calling the feature extractor function to save the features within the dataframe

def preproc_dataset(arr):
    
    features=[]
    for v in arr:
        f=features_extractor(v)
        v = v.replace('.wav', '')
        v=v[54 :] # replace the number (54) in this line with the number of characters to be removed from the beginning of the path of audio file 
        parts = v.split('_')
        rec,name,series,vehicle= parts
      
        features.append({'file': v,
                           'vehicle': vehicle,
                           'series':series,
                           'feature': f,
                           })
        
    df_dataset = pd.DataFrame(mfcc, columns=('file','series', 'vehicle','feature'))      
    return df_dataset

In [ ]:
df_train=preproc_dataset(pth)
df_train=df_train.sample(frac=1).reset_index(drop=True)

In [ ]:
df_train['vehicle'].value_counts()

In [ ]:
z=df_train['feature']
x=z.tolist()
y=np.array(x,dtype=object)
array=np.save('mvd_gfcc.npy',y)

In [ ]:
df_train=df_train.drop(['feature'], axis=1)

df_train.to_csv('mvd_gfcc.csv')